In [1]:
]activate .

  Activating new project at `~/Projects/Noticer.jl`


In [3]:
using PuzzleTools

In [8]:
words = PuzzleTools.Wordsets.unixwords();

In [10]:
using Distributions

In [11]:
using LinearAlgebra: normalize

In [16]:
dist = Multinomial(3, normalize([1, 2, 3], 1))

Multinomial{Float64, Vector{Float64}}(n=3, p=[0.16666666666666666, 0.3333333333333333, 0.5])

In [24]:
pdf(dist, [1, 2, 0])

0.05555555555555554

In [27]:
pdf(dist, [0, 0, 3])

0.12500000000000003

In [28]:
pdf(dist, [1, 0, 2])

0.12500000000000003

In [29]:
dist = Multinomial(5, normalize([1, 2, 3], 1))

Multinomial{Float64, Vector{Float64}}(n=5, p=[0.16666666666666666, 0.3333333333333333, 0.5])

In [30]:
pdf(dist, [0, 0, 5])

0.03125

In [31]:
pdf(dist, [1, 0, 4])

0.05208333333333332

In [32]:
0.5^5

0.03125

In [33]:
logpdf(dist, [1, 0, 4])

-2.954910279033736

In [34]:
dist = Multinomial(5, normalize([1, 0, 2, 3], 1))

Multinomial{Float64, Vector{Float64}}(n=5, p=[0.16666666666666666, 0.0, 0.3333333333333333, 0.5])

In [35]:
pdf(dist, [0, 0, 0, 5])

0.03125

On reflection, I'm not sure the multinomial distribution makes sense here. Let's go back to the idea of counting the number of 'm's in a set of words. It's clearly pretty interesting if all of our words have exactly 2 'm's. What if 4 out of 5 have exactly 2 'm's but the last has some other number? That might be interesting too, but does its interestingness (from a puzzle perspective) depend on how many 'm's that last word has? I'd argue that it doesn't. A count of 'm's of [2, 2, 2, 2, 5] is more unlikely (according to the multinomial distribution) than a count of [2, 2, 2, 2, 1] since few words have 5 'm's, but I don't think that actually matters from a puzzle perspective. In both cases, we have 4 of 5 words with some interesting feature (2 'm's) and 1 word without that feature. 

That seems to argue in favor of the original one-hot style from Collective.jl. In that case, "has 2 'm's" is a boolean feature, and the sample would be [true, true, true, true, false] regardless of whether we had 5 'm's or 1 in the last word. I think that's closer to what we actually want. 

In [55]:
dist = Binomial(5, 0.25)

Binomial{Float64}(n=5, p=0.25)

In [56]:
pdf(dist, 0)

0.23730468750000017

In [57]:
0.75^5

0.2373046875

In [58]:
pdf(dist, 5)

0.0009765625000000009

In [59]:
0.25^5

0.0009765625

In [60]:
pdf(dist, 2)

0.26367187500000017

In [46]:
function naive_pdf(dist, k)
    factorial(dist.n) / (factorial(k) * factorial(dist.n - k)) * dist.p^k * (1 - dist.p)^(dist.n - k)
end

naive_pdf (generic function with 1 method)

In [61]:
dist = Binomial(5, 0.25)

Binomial{Float64}(n=5, p=0.25)

In [62]:
naive_pdf(dist, 2)

0.263671875

In [63]:
using BenchmarkTools

In [64]:
@btime pdf($dist, 2)

  103.809 ns (0 allocations: 0 bytes)


0.26367187500000017

In [65]:
@btime logpdf($dist, 2)

  93.359 ns (0 allocations: 0 bytes)


-1.3330498466010776

In [66]:
@btime naive_pdf($dist, 2)

  7.728 ns (0 allocations: 0 bytes)


0.263671875

In [67]:
dist = Binomial(10, 0.25)

Binomial{Float64}(n=10, p=0.25)

In [68]:
@btime pdf($dist, 2)

  94.074 ns (0 allocations: 0 bytes)


0.28156757354736345

In [69]:
@btime naive_pdf($dist, 2)

  96.182 ns (0 allocations: 0 bytes)


0.2815675735473633

In [70]:
dist = Binomial(20, 0.25)

Binomial{Float64}(n=20, p=0.25)

In [71]:
@btime pdf($dist, 2)

  94.523 ns (0 allocations: 0 bytes)


0.06694780759971757

In [72]:
@btime naive_pdf($dist, 2)

  96.304 ns (0 allocations: 0 bytes)


0.06694780759971763

In [73]:
@btime pdf($dist, 10)

  95.431 ns (0 allocations: 0 bytes)


0.009922275279677708

In [74]:
@btime naive_pdf($dist, 10)

  169.432 ns (0 allocations: 0 bytes)


0.009922275279677706

In [75]:
dist = Binomial(1000, 0.25)

Binomial{Float64}(n=1000, p=0.25)

In [76]:
@btime pdf($dist, 500)

  95.614 ns (0 allocations: 0 bytes)


8.559791450762579e-65

In [77]:
@btime naive_pdf($dist, 500)

LoadError: OverflowError: 1000 is too large to look up in the table; consider using `factorial(big(1000))` instead

In [78]:
@btime Distributions.betalogpdf(1, 2, 3)

  28.904 ns (0 allocations: 0 bytes)


-Inf

In [80]:
@edit Distributions.betalogpdf(1, 2, 3)

In [82]:
using StatsBase

In [83]:
kldivergence([1, 0, 0], [1/3, 1/3, 1/3])

1.0986122886681098

In [84]:
kldivergence([1, 0, 0], normalize([1, 2, 3], 1))

1.791759469228055

In [85]:
kldivergence([0, 0, 1], normalize([1, 2, 3], 1))

0.6931471805599453

In [86]:
kldivergence([1, 0], [0.25, 0.75])

1.3862943611198906

In [96]:
using HypothesisTests: MultinomialLRTest, ChisqTest

In [91]:
MultinomialLRTest([1, 1, 1], normalize([1, 1, 1], 1))

Multinomial Likelihood Ratio Test
---------------------------------
Population details:
    parameter of interest:   Multinomial Probabilities
    value under h_0:         [0.333333, 0.333333, 0.333333]
    point estimate:          [0.333333, 0.333333, 0.333333]
    95% confidence interval: [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           1.0000

Details:
    Sample size:        3
    statistic:          0.0
    degrees of freedom: 2
    residuals:          [0.0, 0.0, 0.0]
    std. residuals:     [0.0, 0.0, 0.0]


In [95]:
MultinomialLRTest([1, 0, 10], normalize([1, 1, 1], 1))

Multinomial Likelihood Ratio Test
---------------------------------
Population details:
    parameter of interest:   Multinomial Probabilities
    value under h_0:         [0.333333, 0.333333, 0.333333]
    point estimate:          [0.0909091, 0.0, 0.909091]
    95% confidence interval: [(0.0, 0.2467), (0.0, 0.1558), (0.8182, 1.0)]

Test summary:
    outcome with 95% confidence: reject h_0
    one-sided p-value:           NaN

Details:
    Sample size:        11
    statistic:          NaN
    degrees of freedom: 2
    residuals:          [-1.39262, -1.91485, 3.30748]
    std. residuals:     [-1.70561, -2.34521, 4.05081]


In [97]:
ChisqTest([1, 1, 1], normalize([1, 1, 1], 1))

Pearson's Chi-square Test
-------------------------
Population details:
    parameter of interest:   Multinomial Probabilities
    value under h_0:         [0.333333, 0.333333, 0.333333]
    point estimate:          [0.333333, 0.333333, 0.333333]
    95% confidence interval: [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]

Test summary:
    outcome with 95% confidence: fail to reject h_0
    one-sided p-value:           1.0000

Details:
    Sample size:        3
    statistic:          0.0
    degrees of freedom: 2
    residuals:          [0.0, 0.0, 0.0]
    std. residuals:     [0.0, 0.0, 0.0]


In [100]:
ChisqTest([0, 1, 5], normalize([1, 1, 1], 1))

Pearson's Chi-square Test
-------------------------
Population details:
    parameter of interest:   Multinomial Probabilities
    value under h_0:         [0.333333, 0.333333, 0.333333]
    point estimate:          [0.0, 0.166667, 0.833333]
    95% confidence interval: [(0.0, 0.2781), (0.0, 0.4448), (0.6667, 1.0)]

Test summary:
    outcome with 95% confidence: reject h_0
    one-sided p-value:           0.0302

Details:
    Sample size:        6
    statistic:          7.0
    degrees of freedom: 2
    residuals:          [-1.41421, -0.707107, 2.12132]
    std. residuals:     [-1.73205, -0.866025, 2.59808]


In [101]:
ChisqTest([1, 4], [0.75, 0.25])

Pearson's Chi-square Test
-------------------------
Population details:
    parameter of interest:   Multinomial Probabilities
    value under h_0:         [0.75, 0.25]
    point estimate:          [0.2, 0.8]
    95% confidence interval: [(0.0, 0.5257), (0.6, 1.0)]

Test summary:
    outcome with 95% confidence: reject h_0
    one-sided p-value:           0.0045

Details:
    Sample size:        5
    statistic:          8.066666666666666
    degrees of freedom: 1
    residuals:          [-1.42009, 2.45967]
    std. residuals:     [-2.84019, 2.84019]


In [107]:
dist = Binomial(5, 0.25)
pdf(dist, 4) + pdf(dist, 5)

0.015625000000000007

In [110]:
using HypothesisTests: pvalue

In [111]:
pvalue(ChisqTest([0, 0, 5, 0], normalize([1, 3, 1, 0.5], 1)))

5.133013955787431e-5

In [112]:
pvalue(ChisqTest([0, 0, 4, 1], normalize([1, 3, 1, 0.5], 1)))

0.001995790028753647